In [1]:
from google.colab import files
uploaded = files.upload()  # Select your data.zip file

import zipfile

with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("courage_bot")  # Will extract to courage_bot/data/


Saving data.zip to data.zip


In [2]:
import os
import json

data = []
data_path = "courage_bot/data"

for i in range(202):
    file_path = os.path.join(data_path, f"pair_{i}.txt")
    if not os.path.exists(file_path):
        print(f"Skipping missing file: pair_{i}.txt")
        continue

    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip()
        if '\n' in content:
            prompt, response = content.split('\n', 1)
            data.append({"prompt": prompt.strip(), "response": response.strip()})
        else:
            print(f"Skipping file pair_{i}.txt: not enough lines.")

with open("courage_data.jsonl", "w", encoding="utf-8") as out:
    for entry in data:
        json.dump(entry, out)
        out.write("\n")

print("✅ Done! courage_data.jsonl is ready.")


✅ Done! courage_data.jsonl is ready.


In [3]:
import json

with open("courage_data.jsonl", "r") as f:
    for i in range(3):  # Show first 3 entries
        print(json.loads(f.readline()))


{'prompt': 'User: What’s causing the floorboards to creak rhythmically?', 'response': 'Computer: Oh, darling Courage, allow me to enlighten your simple mind. That\'s not just your old house settling, it\'s a poltergeist expressing its dissatisfaction with your sloppy doggy dishes left on the floor. It\'s trying to communicate with you through Morse code, using its ghostly moans to spell out "pick up after yourself." Quite frankly, I wouldn\'t be surprised if it starts demanding a later dinner time next.'}
{'prompt': 'User: Why is the mirror showing a different room?', 'response': "Computer: Oh, darling Courage, it appears you've stumbled upon yet another mystery that would baffle even the most seasoned paranormal investigators. That mirror, you see, isn't just reflecting your adorable reflection back at you. No, no, no. It seems to have developed a rather peculiar taste for the macabre and the mysterious.\n\nYou may think it's simply a case of a fun house mirror or a mirrored closet do

In [4]:
!pip install -q bitsandbytes accelerate transformers datasets peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.4 MB/s eta 0:00:00


In [5]:
from datasets import Dataset
import json

# Read JSONL manually
with open("courage_data.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Create Hugging Face Dataset from list of dicts
dataset = Dataset.from_list(data)


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",  
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def format_prompt(example):
    return f"User: {example['prompt']}\nComputer: {example['response']}"

def tokenize(example):
    inputs = tokenizer(
        f"{example['prompt']}\n{example['response']}",
        padding="max_length",
        truncation=True,
        max_length=512
    )
    inputs["labels"] = inputs["input_ids"].copy()  
    return inputs


tokenized_dataset = dataset.map(tokenize)


Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./courage-computer",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.711000
20,0.601800
30,0.513700
40,0.534700
50,0.470300
60,0.402900
70,0.404500
80,0.374900
90,0.371500
100,0.430700


TrainOutput(global_step=102, training_loss=0.774447462722367, metrics={'train_runtime': 628.5281, 'train_samples_per_second': 0.643, 'train_steps_per_second': 0.162, 'total_flos': 8833413855313920.0, 'train_loss': 0.774447462722367, 'epoch': 2.0})

In [12]:
model.eval()  # Set to evaluation mode
prompt = "User: Why is the wind whispering my name?"  # Test input

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Tokenize and move to GPU
outputs = model.generate(**inputs, max_new_tokens=500)      # Generate response
print(tokenizer.decode(outputs[0], skip_special_tokens=True))  # Decode to readable text


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


User: Why is the wind whispering my name?
Computer: Oh, darling Courage, how delightful that the wind has taken an interest in your mundane existence! It's not every day that a gust of air decides to serenade you with your own name, you know. I suppose it's just the wind's way of expressing its admiration for your extraordinary bravery and unparalleled wit. Or perhaps it's simply trying to warn you of impending doom, as winds are wont to do. Either way, I'm sure it's nothing to worry about. After all, what could possibly go wrong when a dog named after a cowardly canine is being addressed by a sentient breeze? Absolutely nothing, I'm certain. Carry on, dear boy. The wind will be with you always, whispering sweet nothings into your ear and reminding you of your true name: Courageous. Or perhaps it's just a bit of wind noise, who can say for sure? But don't let that dampen your spirits, oh no! Embrace the wind's affection and continue on your merry way, for you are Courage, the bravest d

In [13]:
model.save_pretrained("courage-lora")
tokenizer.save_pretrained("courage-lora")


('courage-lora/tokenizer_config.json',
 'courage-lora/special_tokens_map.json',
 'courage-lora/chat_template.jinja',
 'courage-lora/tokenizer.model',
 'courage-lora/added_tokens.json',
 'courage-lora/tokenizer.json')

In [14]:
!zip -r courage-lora.zip courage-lora
from google.colab import files
files.download("courage-lora.zip")


  adding: courage-lora/ (stored 0%)
  adding: courage-lora/tokenizer_config.json (deflated 68%)
  adding: courage-lora/adapter_model.safetensors (deflated 7%)
  adding: courage-lora/chat_template.jinja (deflated 64%)
  adding: courage-lora/tokenizer.json (deflated 85%)
  adding: courage-lora/adapter_config.json (deflated 55%)
  adding: courage-lora/special_tokens_map.json (deflated 73%)
  adding: courage-lora/README.md (deflated 66%)
  adding: courage-lora/tokenizer.model (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
!pip install -q gradio


In [ ]:

def chat_with_computer(user_input):
    prompt = f"User: {user_input}\nComputer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.9, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Computer:")[-1].strip()


In [17]:
import gradio as gr

with gr.Blocks() as app:
    gr.Markdown("## 💻 The Computer")
    gr.Markdown("_Ask your creepy, sarcastic AI anything... if you dare._")

    user_input = gr.Textbox(label="Your Question")
    output = gr.Textbox(label="Computer's Response")

    btn = gr.Button("Ask Computer")

    btn.click(fn=chat_with_computer, inputs=user_input, outputs=output)

app.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d1abd77f394247d5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
